In [19]:
import json
import pandas as pd
import numpy as np
from statistics import mean
import glob

In [20]:
def extract_fd_vals(json_file):
    f = open(json_file)
    data = json.load(f)
    fd_data = data['fd_mean'] 
    f.close() 
    return fd_data

In [21]:
def mean_fd_val(sub):
    fd_list = []
    for json_file in glob.glob(f'../derivatives/mriqc/derivatives/sub-{sub}*_bold.json'):
        fd_list.append(extract_fd_vals(json_file))
    return np.mean(fd_list)

def trial_fd_val(sub, run, condition, block):
    y = glob.glob(f'../derivatives/mriqc/derivatives/sub-{subnum}_task-sr{condition}{block}_run-0{run}_bold.json')
    #print(sub)
    return  extract_fd_vals(y[0])

In [22]:
test_file = pd.read_csv('test_output.csv')

In [23]:
print(test_file)

                     Unnamed: 0  Starting Volume  Length in Volumes  \
0     3847_doors__run_1_block_a                0                 66   
1     3847_doors__run_1_block_b               66                 64   
2     3847_doors__run_2_block_a                0                 66   
3     3847_doors__run_2_block_b               66                 64   
4    3847_social__run_1_block_a                0                 66   
..                          ...              ...                ...   
235   3890_doors__run_2_block_b               66                 64   
236  3890_social__run_1_block_a                0                 66   
237  3890_social__run_1_block_b               66                 64   
238  3890_social__run_2_block_a                0                 66   
239  3890_social__run_2_block_b               66                 64   

    Long or Short Block Type Condition  Run Number  N Volumes  \
0            long          a     doors           1  66.333333   
1           short

In [26]:
test_file_ordering = []
    
for row in test_file.iterrows():
    run = row[1][6]
    subnum = row[1][0][:4]
    condition = row[1][5]
    condition = condition.capitalize()
    valence = row[1][8]
    block = row[1][4]
    
    mean_fd_sub = mean_fd_val(subnum)
    
    mean_fd_run = trial_fd_val(subnum, run, condition, block)
    
    fd = (mean_fd_run - mean_fd_sub)
    
    test_file_ordering.append([subnum, f"L1_task-sr{condition}{block}_model-01_type-act_run-0{run}_sm-6_variant-dctAROMAnonaggr.feat", valence, fd])
            
    df = pd.DataFrame(data = test_file_ordering, columns = ['subject', 'L1', 'inst. valence', 'demeaned fd'])

    
    
per_sub = np.array_split(df, 30)

In [62]:
for i in per_sub[8:9]:
    sub = i.iloc[0][0]
    
    
    sorted_vals = i.sort_values(by = 'inst. valence')
    
    l1_models = sorted_vals.iloc[0:, 1].tolist()
    l1_command = " ".join([j for j in l1_models])
    print(sub, l1_command)
    
    #sorted_vals = i.sort_values(by = 'inst. valence')
 
    
    #fd_list = sorted_vals.iloc[0:, 3].to_list()
    #fd_command = " ".join([str(j) for j in fd_list])
    #print(sub, len(l1_models) == 8)
    #!echo ./models/L2_combine_socialDoors.sh $sub  $l1_command $fd_command

3886 L1_task-srDoorsa_model-01_type-act_run-01_sm-6_variant-dctAROMAnonaggr.feat L1_task-srDoorsb_model-01_type-act_run-02_sm-6_variant-dctAROMAnonaggr.feat L1_task-srSocialb_model-01_type-act_run-01_sm-6_variant-dctAROMAnonaggr.feat L1_task-srSociala_model-01_type-act_run-02_sm-6_variant-dctAROMAnonaggr.feat L1_task-srDoorsb_model-01_type-act_run-01_sm-6_variant-dctAROMAnonaggr.feat L1_task-srDoorsa_model-01_type-act_run-02_sm-6_variant-dctAROMAnonaggr.feat L1_task-srSociala_model-01_type-act_run-01_sm-6_variant-dctAROMAnonaggr.feat L1_task-srSocialb_model-01_type-act_run-02_sm-6_variant-dctAROMAnonaggr.feat


In [ ]:
#write out dfs 

for i in per_sub[0:1]:
    sub = i.iloc[0][0]
    #i.to_csv(f"../derivatives/l2_tests/{i.iloc[0][0]}.csv", index = False)
    
    sorted_vals = i.sort_values(by = 'inst. valence')
    l1_models = sorted_vals.iloc[0:, 1].tolist()
    l1_command = " ".join([j for j in l1_models])
    
    
    sorted_vals = i.sort_values(by = 'inst. valence')
 
    
    fd_list = sorted_vals.iloc[0:, 3].to_list()
    fd_command = " ".join([str(j) for j in fd_list])
    print(sub, len(l1_models) == 8)
    !echo ./models/L2_combine_socialDoors.sh $sub  $l1_command $fd_command